In [1]:
import numpy as np
import pandas as pd
import os
import librosa
import librosa.display
from pydub.silence import split_on_silence
from pydub import AudioSegment, effects 
from scipy.io.wavfile import read, write

C:\Users\Kavya Duvvuri\anaconda3\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


def preprocess_audio(audio_file_name):

    audio_file, sr = librosa.load(audio_file_name)
    
    audio_file = librosa.effects.preemphasis(audio_file)
    audio_file = librosa.effects.trim(audio_file, top_db=20)[0]
    audio_file = librosa.util.normalize(audio_file)
    
    return sr, audio_file

In [3]:
DATA_PATH = r"D:/Datasets/Speech Processing/Data/"

def get_file_names(path = DATA_PATH):
    file_names = os.listdir(path)
    return file_names

def process_audio(audio_file_name):
    rate, audio = read(f"./Data/{audio_file_name}")

    aud = AudioSegment(audio.tobytes(), frame_rate = rate,
                         sample_width = audio.dtype.itemsize, channels = 1)

    audio_chunks = split_on_silence(
        aud,
        min_silence_len = 1000,
        silence_thresh = -50,
        keep_silence = 500,)
    
    # audio chunks are combined here
    audio_processed = sum(audio_chunks)
    audio_processed = np.array(audio_processed.get_array_of_samples())
    audio_file = audio_file_name.split(".")[0]
    write(f"./Processed Data/{audio_file}_PROCESSED.wav", rate, audio_processed)

file_names = get_file_names()
for file_name in file_names:
    process_audio(file_name)
print("Done")

In [4]:
def get_label_dict(df, file_list, test=False):
    label_dict = {}
    
    if test:
        df_list = df['PHQ_Score']
    else:
        df_list = df['PHQ8_Score']

    for file in file_list:
        patient_num = int(file.split("/")[-1].split("_")[0])

        patient_list = list(df['Participant_ID'])

        idx = patient_list.index(patient_num)

        phq8_score = int(df_list[idx])
        
        if phq8_score in range(0, 6):
            score = 0
        elif phq8_score in range(6, 15):
            score = 1
        elif phq8_score in range(15, 22):
            score = 2
        elif phq8_score in range(22, 28):
            score = 3

        label_dict[f"{file}"] = score
    
    return label_dict

In [5]:
def get_set(df):
    df_files = [f"D:/Datasets/Speech Processing/Processed Data/{x}_AUDIO_PROCESSED.wav" for x in df['Participant_ID']]

    return list(set(df_files))

In [6]:
train_df, test_df, val_df = pd.read_csv(f"./train.csv"), pd.read_csv(f"./test.csv"), pd.read_csv(f"./val.csv")

train, test, val = get_set(train_df), get_set(test_df), get_set(val_df)

In [7]:
train_dict, test_dict, val_dict = get_label_dict(train_df, train), get_label_dict(test_df, test, True), get_label_dict(val_df, val)
data_dict = {**train_dict, **test_dict, **val_dict}

In [8]:
PROCESSED_DATA_PATH = "D:/Datasets/Speech Processing/Processed Data/"

In [9]:
data_list = list(data_dict.keys())
label_list = list(data_dict.values())

In [10]:
new_data_list = []
new_label_list = []
for data in data_list:
    audios = os.listdir(PROCESSED_DATA_PATH)
    data_2 = data.split('/')[2]
    if data_2 in audios:
        new_data_list.append(data)
        new_label_list.append(label_list[data_list.index(data)])

In [18]:
csv_path = "./Data Labels.csv"
data_df = pd.read_csv(csv_path)
new_data = list(data_df["New Data"])
new_labels = list(data_df["New Labels"])

In [19]:
csv_path = "./Combined Features.csv"
combined_df = pd.read_csv(csv_path)
combined = list(combined_df["Combined"])
combined_features = []
for i in combined:
    new_i = i.replace('\n', ' ')
    new_i = new_i.replace('[', '')
    new_i = new_i.replace(']', '')
    array_data = np.fromstring(new_i, dtype=np.float64, sep=' ')
    combined_features.append(array_data)
print(len(combined_features))

744


In [20]:
X = np.array(combined_features).reshape(744, 90, 1)

In [24]:
Y = np.array(new_labels)

In [25]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential

In [26]:
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size = 0.20)

In [27]:
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(90,1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Conv1D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='softmax'))

In [28]:
from sklearn.model_selection import StratifiedKFold
k = 5

# Perform k-fold cross-validation
skf = StratifiedKFold(n_splits=k, shuffle=True)
fold = 1
accuracy_scores = []

for train_index, val_index in skf.split(X, Y):
    print(f"Training on Fold: {fold}")
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = Y[train_index], Y[val_index]
    
    # Compile and train the model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=32, epochs=200, verbose=1)
    
    # Evaluate the model on the validation set
    _, accuracy = model.evaluate(X_val, y_val, verbose=0)
    accuracy_scores.append(accuracy)
    
    print(f"Accuracy on Fold {fold}: {accuracy}")
    print("------------------------------")
    fold += 1

# Calculate the average accuracy across all folds
avg_accuracy = np.mean(accuracy_scores)
print("Average Accuracy:", avg_accuracy)

Training on Fold: 1
Epoch 1/200
19/19 [==============================] - 3s 14ms/step - loss: 2.1943 - accuracy: 0.4034
Epoch 2/200
19/19 [==============================] - 0s 14ms/step - loss: 1.2732 - accuracy: 0.4370
Epoch 3/200
19/19 [==============================] - 0s 15ms/step - loss: 1.1597 - accuracy: 0.4571
Epoch 4/200
19/19 [==============================] - 0s 15ms/step - loss: 1.1083 - accuracy: 0.4521
Epoch 5/200
19/19 [==============================] - 0s 15ms/step - loss: 1.0822 - accuracy: 0.4639
Epoch 6/200
19/19 [==============================] - 0s 14ms/step - loss: 1.0655 - accuracy: 0.4840
Epoch 7/200
19/19 [==============================] - 0s 13ms/step - loss: 1.0553 - accuracy: 0.4824
Epoch 8/200
19/19 [==============================] - 0s 14ms/step - loss: 1.0657 - accuracy: 0.4706
Epoch 9/200
19/19 [==============================] - 0s 15ms/step - loss: 1.0480 - accuracy: 0.4756
Epoch 10/200
19/19 [==============================] - 0s 14ms/step - loss: 1.052

19/19 [==============================] - 0s 13ms/step - loss: 0.9550 - accuracy: 0.5345
Epoch 83/200
19/19 [==============================] - 0s 13ms/step - loss: 0.9483 - accuracy: 0.5277
Epoch 84/200
19/19 [==============================] - 0s 13ms/step - loss: 0.9379 - accuracy: 0.5681
Epoch 85/200
19/19 [==============================] - 0s 13ms/step - loss: 0.9310 - accuracy: 0.5479
Epoch 86/200
19/19 [==============================] - 0s 13ms/step - loss: 0.9565 - accuracy: 0.5345
Epoch 87/200
19/19 [==============================] - 0s 13ms/step - loss: 0.9462 - accuracy: 0.5294
Epoch 88/200
19/19 [==============================] - 0s 13ms/step - loss: 0.9292 - accuracy: 0.5630
Epoch 89/200
19/19 [==============================] - 0s 12ms/step - loss: 0.9338 - accuracy: 0.5647
Epoch 90/200
19/19 [==============================] - 0s 13ms/step - loss: 0.9512 - accuracy: 0.5429
Epoch 91/200
19/19 [==============================] - 0s 13ms/step - loss: 0.9419 - accuracy: 0.5412
Epo

19/19 [==============================] - 0s 14ms/step - loss: 0.7731 - accuracy: 0.6420
Epoch 163/200
19/19 [==============================] - 0s 14ms/step - loss: 0.7567 - accuracy: 0.6521
Epoch 164/200
19/19 [==============================] - 0s 13ms/step - loss: 0.7645 - accuracy: 0.6437
Epoch 165/200
19/19 [==============================] - 0s 15ms/step - loss: 0.7346 - accuracy: 0.6605
Epoch 166/200
19/19 [==============================] - 0s 13ms/step - loss: 0.7501 - accuracy: 0.6538
Epoch 167/200
19/19 [==============================] - 0s 14ms/step - loss: 0.7631 - accuracy: 0.6437
Epoch 168/200
19/19 [==============================] - 0s 13ms/step - loss: 0.7782 - accuracy: 0.6521
Epoch 169/200
19/19 [==============================] - 0s 13ms/step - loss: 0.7406 - accuracy: 0.6790
Epoch 170/200
19/19 [==============================] - 0s 14ms/step - loss: 0.7855 - accuracy: 0.6286
Epoch 171/200
19/19 [==============================] - 0s 13ms/step - loss: 0.7379 - accuracy: 0

19/19 [==============================] - 0s 13ms/step - loss: 0.6648 - accuracy: 0.7160
Epoch 43/200
19/19 [==============================] - 0s 13ms/step - loss: 0.6786 - accuracy: 0.7076
Epoch 44/200
19/19 [==============================] - 0s 13ms/step - loss: 0.6605 - accuracy: 0.7143
Epoch 45/200
19/19 [==============================] - 0s 12ms/step - loss: 0.6641 - accuracy: 0.6992
Epoch 46/200
19/19 [==============================] - 0s 12ms/step - loss: 0.6588 - accuracy: 0.6807
Epoch 47/200
19/19 [==============================] - 0s 12ms/step - loss: 0.6653 - accuracy: 0.7008
Epoch 48/200
19/19 [==============================] - 0s 12ms/step - loss: 0.6495 - accuracy: 0.7059
Epoch 49/200
19/19 [==============================] - 0s 12ms/step - loss: 0.6569 - accuracy: 0.7042
Epoch 50/200
19/19 [==============================] - 0s 12ms/step - loss: 0.6652 - accuracy: 0.6958
Epoch 51/200
19/19 [==============================] - 0s 12ms/step - loss: 0.6324 - accuracy: 0.7261
Epo

19/19 [==============================] - 0s 13ms/step - loss: 0.4971 - accuracy: 0.7832
Epoch 124/200
19/19 [==============================] - 0s 13ms/step - loss: 0.5347 - accuracy: 0.7647
Epoch 125/200
19/19 [==============================] - 0s 14ms/step - loss: 0.4901 - accuracy: 0.7782
Epoch 126/200
19/19 [==============================] - 0s 14ms/step - loss: 0.5336 - accuracy: 0.7479
Epoch 127/200
19/19 [==============================] - 0s 13ms/step - loss: 0.5032 - accuracy: 0.7798
Epoch 128/200
19/19 [==============================] - 0s 14ms/step - loss: 0.5279 - accuracy: 0.7714
Epoch 129/200
19/19 [==============================] - 0s 14ms/step - loss: 0.5358 - accuracy: 0.7647
Epoch 130/200
19/19 [==============================] - 0s 14ms/step - loss: 0.5262 - accuracy: 0.7664
Epoch 131/200
19/19 [==============================] - 0s 13ms/step - loss: 0.4756 - accuracy: 0.7782
Epoch 132/200
19/19 [==============================] - 0s 13ms/step - loss: 0.5061 - accuracy: 0

...

19/19 [==============================] - 0s 13ms/step - loss: 0.3488 - accuracy: 0.8370
Epoch 44/200
19/19 [==============================] - 0s 12ms/step - loss: 0.3395 - accuracy: 0.8504
Epoch 45/200
19/19 [==============================] - 0s 12ms/step - loss: 0.3261 - accuracy: 0.8639
Epoch 46/200
19/19 [==============================] - 0s 12ms/step - loss: 0.3335 - accuracy: 0.8403
Epoch 47/200
19/19 [==============================] - 0s 12ms/step - loss: 0.3612 - accuracy: 0.8370
Epoch 48/200
19/19 [==============================] - 0s 12ms/step - loss: 0.3431 - accuracy: 0.8504
Epoch 49/200
19/19 [==============================] - 0s 12ms/step - loss: 0.3443 - accuracy: 0.8454
Epoch 50/200
19/19 [==============================] - 0s 12ms/step - loss: 0.3555 - accuracy: 0.8487
Epoch 51/200
19/19 [==============================] - 0s 12ms/step - loss: 0.3446 - accuracy: 0.8319
Epoch 52/200
19/19 [==============================] - 0s 12ms/step - loss: 0.3395 - accuracy: 0.8471
Epo

19/19 [==============================] - 0s 15ms/step - loss: 0.3240 - accuracy: 0.8521
Epoch 125/200
19/19 [==============================] - 0s 13ms/step - loss: 0.3206 - accuracy: 0.8555
Epoch 126/200
19/19 [==============================] - 0s 12ms/step - loss: 0.3424 - accuracy: 0.8437
Epoch 127/200
19/19 [==============================] - 0s 12ms/step - loss: 0.3313 - accuracy: 0.8471
Epoch 128/200
19/19 [==============================] - 0s 12ms/step - loss: 0.3047 - accuracy: 0.8571
Epoch 129/200
19/19 [==============================] - 0s 12ms/step - loss: 0.3116 - accuracy: 0.8555
Epoch 130/200
19/19 [==============================] - 0s 12ms/step - loss: 0.3137 - accuracy: 0.8521
Epoch 131/200
19/19 [==============================] - 0s 12ms/step - loss: 0.3216 - accuracy: 0.8487
Epoch 132/200
19/19 [==============================] - 0s 12ms/step - loss: 0.3415 - accuracy: 0.8437
Epoch 133/200
19/19 [==============================] - 0s 12ms/step - loss: 0.3302 - accuracy: 0

19/19 [==============================] - 0s 13ms/step - loss: 0.3680 - accuracy: 0.8490
Epoch 4/200
19/19 [==============================] - 0s 12ms/step - loss: 0.3426 - accuracy: 0.8423
Epoch 5/200
19/19 [==============================] - 0s 12ms/step - loss: 0.3234 - accuracy: 0.8406
Epoch 6/200
19/19 [==============================] - 0s 12ms/step - loss: 0.3306 - accuracy: 0.8456
Epoch 7/200
19/19 [==============================] - 0s 12ms/step - loss: 0.3284 - accuracy: 0.8507
Epoch 8/200
19/19 [==============================] - 0s 12ms/step - loss: 0.3438 - accuracy: 0.8456
Epoch 9/200
19/19 [==============================] - 0s 13ms/step - loss: 0.3337 - accuracy: 0.8372
Epoch 10/200
19/19 [==============================] - 0s 13ms/step - loss: 0.3388 - accuracy: 0.8540
Epoch 11/200
19/19 [==============================] - 0s 12ms/step - loss: 0.3056 - accuracy: 0.8691
Epoch 12/200
19/19 [==============================] - 0s 12ms/step - loss: 0.3103 - accuracy: 0.8406
Epoch 13/

19/19 [==============================] - 0s 13ms/step - loss: 0.3355 - accuracy: 0.8456
Epoch 85/200
19/19 [==============================] - 0s 12ms/step - loss: 0.2969 - accuracy: 0.8523
Epoch 86/200
19/19 [==============================] - 0s 13ms/step - loss: 0.3110 - accuracy: 0.8574
Epoch 87/200
19/19 [==============================] - 0s 13ms/step - loss: 0.2867 - accuracy: 0.8758
Epoch 88/200
19/19 [==============================] - 0s 13ms/step - loss: 0.2988 - accuracy: 0.8507
Epoch 89/200
19/19 [==============================] - 0s 13ms/step - loss: 0.3050 - accuracy: 0.8607
Epoch 90/200
19/19 [==============================] - 0s 13ms/step - loss: 0.3165 - accuracy: 0.8641
Epoch 91/200
19/19 [==============================] - 0s 12ms/step - loss: 0.2936 - accuracy: 0.8775
Epoch 92/200
19/19 [==============================] - 0s 12ms/step - loss: 0.3194 - accuracy: 0.8591
Epoch 93/200
19/19 [==============================] - 0s 12ms/step - loss: 0.4118 - accuracy: 0.8658
Epo

19/19 [==============================] - 0s 14ms/step - loss: 0.3225 - accuracy: 0.8557
Epoch 165/200
19/19 [==============================] - 0s 13ms/step - loss: 0.2978 - accuracy: 0.8473
Epoch 166/200
19/19 [==============================] - 0s 12ms/step - loss: 0.2994 - accuracy: 0.8591
Epoch 167/200
19/19 [==============================] - 0s 12ms/step - loss: 0.3025 - accuracy: 0.8473
Epoch 168/200
19/19 [==============================] - 0s 13ms/step - loss: 0.2999 - accuracy: 0.8641
Epoch 169/200
19/19 [==============================] - 0s 12ms/step - loss: 0.3062 - accuracy: 0.8691
Epoch 170/200
19/19 [==============================] - 0s 13ms/step - loss: 0.2786 - accuracy: 0.8591
Epoch 171/200
19/19 [==============================] - 0s 13ms/step - loss: 0.2832 - accuracy: 0.8725
Epoch 172/200
19/19 [==============================] - 0s 13ms/step - loss: 0.3182 - accuracy: 0.8574
Epoch 173/200
19/19 [==============================] - 0s 14ms/step - loss: 0.3364 - accuracy: 0